In [1]:
# import expiration data
import csv
reader = csv.DictReader(open("/Users/jefferychen/Desktop/RecipeRecommendation/expiration_data.csv"))
expiration = []
for line in reader:
    expiration.append(line)
for i in range(len(expiration)):
    expiration[i] = dict(expiration[i])
    
# import recipe
import csv
reader = csv.DictReader(open("/Users/jefferychen/Desktop/RecipeRecommendation/recipe.csv"))
recipe = []
for line in reader:
    recipe.append(line)
for i in range(len(recipe)):
    recipe[i] = dict(recipe[i])

# import foodlist
foodList = open("/Users/jefferychen/Desktop/RecipeRecommendation/foodList.txt")
foodList = foodList.readlines()
for i in range(len(foodList)):
    foodList[i] = foodList[i][:-1]

In [2]:
len(recipe)

20111

In [3]:
#from pattern.text.en import singularize
# user: list of dictionaries
# epiration: list of dictionaries
# destructively append "expiration date" to each dictionary in "user" 
def addExpirationDates(user, expiration):
    for userItem in user[1:]:
        curr = userItem['name']
        curr_split = curr.split(' ')
        curr_max = [0, float('inf')]
        exactMatched = False
        for expItem in expiration:
            
            # exact match
            if curr == expItem['name']:
                userItem['expireIn'] = int(expItem['expireIn'])
                exactMatched = True
                break
                
            # split into set and find the maximum number of names that also exist in expiration data
            expItem_split = expItem['name'].split(' ')
            # calculate curr overlap score
            overlapped = 0
            for i in curr_split:
                if i in expItem_split:
                    overlapped += 1
            if overlapped > curr_max[0]:
                curr_max = [overlapped, int(expItem['expireIn'])]
            elif overlapped == curr_max[0]:
                curr_max[1] = min(curr_max[1], int(expItem['expireIn']))
        if curr_max[0] > 0:
            userItem['expireIn'] = curr_max[1]
        # if no matching found, set -1 as indicator value
        elif not exactMatched:
            userItem['expireIn'] = -1

In [4]:
def generate_parsed(item, foodList):
    results = []
    if item in foodList:
        results += [item]
    else:
        for w in item.split(' '):
            if w in foodList:
                results += [w]
    return list(set(results))

In [5]:
def addParsedUserData(user, foodList):
    for item in user[1:]:
        item['parsed_name'] = generate_parsed(item['name'], foodList)

In [6]:
def calculateOverlapScore(user, recipe_item):
    score = 0
    for userItem in user[1:]:
        for i in userItem['parsed_name']:
            if i in recipe_item['ingredients_processed']:
                score += 1
    return score

In [7]:
def calculateExpireScore(user, recipe_item):
    score = 0
    for userItem in user[1:]:
        for i in userItem['parsed_name']:
            if i in recipe_item['ingredients_processed']:
                score += (1 / userItem['expireIn']) ** 2
    return score

In [8]:
# due to replicate recipe in recipe data, remove replicate item in top_5
def checkRepeated(recipe_item, curr_list):
    for i in curr_list:
        if recipe_item['title'] == i[0]['title']:
            return True
    return False

In [9]:
def top_5(user, recipe):
    top_5 = []
    min_score = float('inf')
    min_index = 0
    for recipe_item in recipe:
        if checkRepeated(recipe_item, top_5):
            continue
        curr_score = calculateOverlapScore(user, recipe_item) + 10 * calculateExpireScore(user, recipe_item)
        if len(top_5) < 5:
            top_5 += [[recipe_item, curr_score]]
            if curr_score < min_score:
                min_score = curr_score
                min_index = len(top_5) - 1
        else:
            if curr_score > min_score:
                top_5[min_index] = [recipe_item, curr_score]
                new_min = float('inf')
                for i in range(5):
                    if top_5[i][1] < new_min:
                        new_min = top_5[i][1]
                        min_index = i
                min_score = new_min
    return top_5

In [10]:
# create user data dictionary given a list of scanned food names and return user data
def createUserDict(names):
    user = []
    user.append({'profile': 'N/A'}) #when dealing with foods, need to use uer[1:] since the first dictionary is profile
    for name in names:
        user += [{'name': name}]
    addExpirationDates(user, expiration)
    addParsedUserData(user, foodList)
    return user

In [11]:
import copy
def continuousTop5(user, recipe):
    #nonlocal recipe_copy
    recipe_copy = copy.deepcopy(recipe)
    def g():
        curr = top_5(user, recipe_copy)
        for i in curr:
            recipe_copy.remove(i[0])
        return curr
    return g

In [12]:
# example usage: input a list of scanned food names (l), create the processed user data by calling createUserDict, and call top_5 to retrieve the top 5 recipe recommendations
l = ['olive', 'cheese', 'mozzarella', 'flour', 'pepperoni', 'sausage', 'yeast', 'ham', 'dough', 'pineapple', 'artichoke']
user = createUserDict(l)
recipe_generator = continuousTop5(user, recipe)
# return a list of lists where each item is a recipe and score pair (format: [recipe, score], where recipe is a dictionary and score is the recommendation score)
curr_5 = recipe_generator()

In [13]:
user

[{'profile': 'N/A'},
 {'name': 'olive', 'expireIn': 60, 'parsed_name': ['olive']},
 {'name': 'cheese', 'expireIn': 7, 'parsed_name': ['cheese']},
 {'name': 'mozzarella', 'expireIn': 360, 'parsed_name': ['mozzarella']},
 {'name': 'flour', 'expireIn': 30, 'parsed_name': []},
 {'name': 'pepperoni', 'expireIn': -1, 'parsed_name': ['pepperoni']},
 {'name': 'sausage', 'expireIn': 1, 'parsed_name': ['sausage']},
 {'name': 'yeast', 'expireIn': 5, 'parsed_name': ['yeast']},
 {'name': 'ham', 'expireIn': 3, 'parsed_name': ['ham']},
 {'name': 'dough', 'expireIn': -1, 'parsed_name': []},
 {'name': 'pineapple', 'expireIn': 720, 'parsed_name': ['pineapple']},
 {'name': 'artichoke', 'expireIn': 2, 'parsed_name': ['artichoke']}]

In [14]:
def find_category(s):
    result = []
    s = s[1:s.find(']')]
    s = s.replace('"', "'")
    front = 0
    while front < len(s):
        s = s[front+1:]
        end = s.find("'")
        result.append(s[0:end])
        front = end + 3
    return result

In [15]:
def add_category(user, recipe):
    category = recipe['categories']
    user[0]['profile'] = find_category(category)

In [16]:
add_category(user, recipe[0])

In [17]:
user

[{'profile': ['Sandwich',
   'Bean',
   'Fruit',
   'Tomato',
   'turkey',
   'Vegetable',
   'Kid-Friendly',
   'Apple',
   'Lentil',
   'Lettuce',
   'Cookie']},
 {'name': 'olive', 'expireIn': 60, 'parsed_name': ['olive']},
 {'name': 'cheese', 'expireIn': 7, 'parsed_name': ['cheese']},
 {'name': 'mozzarella', 'expireIn': 360, 'parsed_name': ['mozzarella']},
 {'name': 'flour', 'expireIn': 30, 'parsed_name': []},
 {'name': 'pepperoni', 'expireIn': -1, 'parsed_name': ['pepperoni']},
 {'name': 'sausage', 'expireIn': 1, 'parsed_name': ['sausage']},
 {'name': 'yeast', 'expireIn': 5, 'parsed_name': ['yeast']},
 {'name': 'ham', 'expireIn': 3, 'parsed_name': ['ham']},
 {'name': 'dough', 'expireIn': -1, 'parsed_name': []},
 {'name': 'pineapple', 'expireIn': 720, 'parsed_name': ['pineapple']},
 {'name': 'artichoke', 'expireIn': 2, 'parsed_name': ['artichoke']}]